In [1]:
import vcfpy
import pandas as pd
import os
import sqlite3

In [2]:
db = sqlite3.connect(
    os.path.join(os.environ['VSC_SCRATCH'], 'test.sqlite'))
vcf = '/lustre1/project/stg_00079/teaching/data/snps.annotated.vcf'

In [3]:
print(f"db  : {db}")
print(f"vcf : {vcf}")

db  : <sqlite3.Connection object at 0x2ac10a80c540>
vcf : /lustre1/project/stg_00079/teaching/data/snps.annotated.vcf


In [15]:
# Create a VCF reader - this parses through the VCF file
reader = vcfpy.Reader.from_path(vcf)

# Empty lists to store the data in
snp_records = []
effect_records = []
call_records = []

# These are the columns from a SNPeff record - easy to later assign column names
effect_rec_names = """snp allele effect impact gene gene_id feature_type feature_id 
                      biotype rank hgvs.c hgvs.p cdna_pos cds_pos prot_pos distance_to_feature
                      messages""".split()
    
# start parsing through the VCF:
for i, record in enumerate(reader):
    
    # we used vt decompose - so I expect NO multiallelic SNPs
    assert len(record.ALT) == 1 
    
    # this is the ALT allele (first and only)
    alt = record.ALT[0]
    
    # compose a SNP name for joins later on
    snp_name = f"{record.CHROM}:{record.POS}:{record.REF}:{alt.value}"
    
    # Store a SNP record - I collect everything now in a list of dictionaries
    # that I will later convert to a Pandas dataframe
    snp_records.append(
        dict(snp = snp_name,
             chrom = record.CHROM,
             pos = record.POS,
             quality = record.QUAL,
             ref = record.REF,
             type = alt.type,
             alt = alt.value))

    # Now I will parse through the individual calls
    for call_record in record.calls:
        
        #name of the sample (TLE66_N or TLE66_T)
        sample = os.path.basename(call_record.sample)
        sample = sample.replace('.bam', '')

        # simple genotype - -1 means not called, otherwise it's the number of ALT alleles observer
        # this helps in later queries 
        # Note - this is not perfect - it does ignore uncalled loci (./.)
        genotype_simple = call_record.data['GT'].count('1')
        
        call_records.append(
            dict(snp = snp_name,
                 sample = sample,
                 genotype = call_record.data['GT'], 
                 genotype_simple = genotype_simple))

    # and parse through all snpEff annotations
    for ann in record.INFO['ANN']:
        ann = ann.split('|')
        # create a dictionary of all fields
        eff_record = dict(zip(effect_rec_names, [snp_name] + ann))
        # convert distance to feature to integer
        try:
            eff_record[intfield] = int(eff_record.get(intfield))
        except:
            eff_record[intfield] = -1
            
        effect_records.append(eff_record)

    #for debugging
    #if i > 4000:
    #    break

#convert lists of dicts to a DataFrame
snp_records = pd.DataFrame.from_records(snp_records)
call_records = pd.DataFrame.from_records(call_records)
effect_records = pd.DataFrame.from_records(effect_records)

#Save to disk
print('snp records', snp_records.to_sql('snp', db, if_exists='replace', index=False))
print('call records', call_records.to_sql('snp_call', db, if_exists='replace', index=False))
print('effect records', effect_records.to_sql('snp_effect', db, if_exists='replace', index=False))


snp records 3505
call records 7010
effect records 19640


In [16]:
pd.read_sql("""SELECT * FROM snp LIMIT 5
            """, db)

,snp,chrom,pos,quality,ref,type,alt
0,chr9:127578816:C:T,chr9,127578816,36.1174,C,SNV,T
1,chr9:127578974:A:G,chr9,127578974,422.7380,A,SNV,G
2,chr9:127579080:A:G,chr9,127579080,172.0220,A,SNV,G
3,chr9:127663498:C:T,chr9,127663498,66.0472,C,SNV,T
4,chr9:127674824:G:T,chr9,127674824,24.6981,G,SNV,T


In [17]:
pd.read_sql("""SELECT * FROM snp_call LIMIT 5
            """, db)

,snp,sample,genotype,genotype_simple
0,chr9:127578816:C:T,TLE66_N,0/1,1
1,chr9:127578816:C:T,TLE66_T,0/1,1
2,chr9:127578974:A:G,TLE66_N,0/1,1
3,chr9:127578974:A:G,TLE66_T,0/1,1
4,chr9:127579080:A:G,TLE66_N,0/1,1


In [20]:
# .T transposes - easier viewing
pd.read_sql("""SELECT * FROM snp_effect LIMIT 5
            """, db).T

,0,1,2,3,4
snp,chr9:127578816:C:T,chr9:127578816:C:T,chr9:127578974:A:G,chr9:127578974:A:G,chr9:127579080:A:G
allele,T,T,G,G,G
effect,upstream_gene_variant,intron_variant,5_prime_UTR_variant,upstream_gene_variant,upstream_gene_variant
impact,MODIFIER,MODIFIER,MODIFIER,MODIFIER,MODIFIER
gene,STXBP1,NIBAN2,NIBAN2,STXBP1,NIBAN2
gene_id,ENSG00000136854,ENSG00000136830,ENSG00000136830,ENSG00000136854,ENSG00000136830
feature_type,transcript,transcript,transcript,transcript,transcript
feature_id,ENST00000637521.2,ENST00000373314.7,ENST00000373314.7,ENST00000637521.2,ENST00000373314.7
biotype,protein_coding,protein_coding,protein_coding,protein_coding,protein_coding
rank,,1/13,1/14,,


#### Find high impact variants

In [22]:
pd.read_sql("""SELECT snp.snp, snp_effect.gene, snp_effect.effect, snp_effect.impact
                 FROM snp, snp_effect
                WHERE snp.snp = snp_effect.snp
                  AND snp_effect.impact == 'HIGH'
                LIMIT 5
            """, db)

,snp,gene,effect,impact
0,chr9:128632018:A:G,SPTAN1,splice_acceptor_variant&intron_variant,HIGH
1,chr9:133257521:T:TC,ABO,splice_acceptor_variant&splice_donor_variant&i...,HIGH
2,chr9:133257521:T:TC,ABO,splice_acceptor_variant&splice_donor_variant&i...,HIGH
3,chr9:135077073:G:GAA,OLFM1,frameshift_variant,HIGH
4,chr9:135077150:GCA:G,OLFM1,frameshift_variant,HIGH


In [48]:
def find_around_gene(gene):
    res = pd.read_sql(f"""SELECT snp.snp, snp_effect.gene, snp_effect.effect, snp_effect.impact
                          FROM snp, snp_effect
                         WHERE snp.snp = snp_effect.snp
                           AND snp_effect.gene == '{gene}'""", db)
    print("no SNPs found:", len(res))
    print(" * per impact:")
    for k, v in res.groupby('impact')['snp'].agg(len).iteritems():
        print(f"  - {v:5} : {k}")
    print(" * per effect:")
    for k, v in res.groupby('effect')['snp'].agg(len).iteritems():
        print(f"  - {v:5} : {k}")
    
find_around_gene('ABO')

no SNPs found: 67
 * per impact:
  -     2 : HIGH
  -    65 : MODIFIER
 * per effect:
  -    14 : downstream_gene_variant
  -    46 : intron_variant
  -     5 : non_coding_transcript_exon_variant
  -     2 : splice_acceptor_variant&splice_donor_variant&intron_variant


In [61]:
pd.read_sql("""SELECT snp.snp, 
                      sc1.sample, sc2.sample, 
                      sc1.genotype, sc2.genotype,
                      snp_effect.gene, snp_effect.effect, snp_effect.impact
                 FROM snp, snp_call as sc1, snp_call as sc2, snp_effect
                WHERE snp.snp = sc1.snp
                  AND snp.snp = sc2.snp
                  AND sc1.genotype != sc2.genotype
                  AND snp.snp = snp_effect.snp
                  AND snp_effect.impact != 'LOW'
                  AND snp_effect.impact != 'MODIFIER'
                LIMIT 5
            """, db)

,snp,sample,sample,genotype,genotype,gene,effect,impact
0,chr9:127802988:A:G,TLE66_N,TLE66_T,1/1,./.,FPGS,missense_variant,MODERATE
1,chr9:127802988:A:G,TLE66_T,TLE66_N,./.,1/1,FPGS,missense_variant,MODERATE
2,chr9:127802988:A:G,TLE66_N,TLE66_T,1/1,./.,FPGS,missense_variant,MODERATE
3,chr9:127802988:A:G,TLE66_T,TLE66_N,./.,1/1,FPGS,missense_variant,MODERATE
4,chr9:127802988:A:G,TLE66_N,TLE66_T,1/1,./.,FPGS,missense_variant,MODERATE
